<a href="https://colab.research.google.com/github/GaelleNovales/Trendr/blob/master/Tweetsearchipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ressources

##Téléchargement des ressources



In [1]:
!pip install nltk
!pip install twint
!pip install the Twython library 
!pip install emoji

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('sentiwordnet')

     |████████████████████████████████| 1.2MB 11.5MB/s 
     |████████████████████████████████| 245kB 25.5MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 153kB 29.8MB/s 
     |████████████████████████████████| 256kB 27.0MB/s 
     |████████████████████████████████| 235kB 29.1MB/s 
  Created wheel for twint: filename=twint-2.1.16-cp36-none-any.whl size=33559 sha256=2c6dcb19307ec4031799642cc4cbaa12d424a6983486e0980b55559af2268fdd
  Stored in directory: /root/.cache/pip/wheels/8b/0c/1d/2f876303c0c9c1cd3591a3fb0ec402081324964218390ef35b
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=78877f03d0adf95355842eced39fbeceb5d522548568deeef891ebb8890bac34
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for googletransx: filename=googletransx-2.4.2-cp36-none-any.whl size=15969 sha256=9cffd61ddec2b29a38d6103de8c11538537

True

##Imports

In [0]:
import twint
from datetime import datetime
import emoji
import regex
import re
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import pandas as pd
from textblob import TextBlob
import csv

#Création du projet

## Importation des tweets selon un hashtag


In [0]:
def export_tweets(mention,taille):
  date = datetime.now()
  date_format = str(date.year)+'-'+str(date.month)+'-'+str(date.day)

  # Configure
  c = twint.Config()
  c.Search = mention
  c.Store_csv = True
  c.Output = c.Search+"_saved.csv"
  c.Limit = taille
  c.Since = date_format
  c.Lang = 'en'
  c.Pandas = True

  # Run
  twint.run.Search(c)

  twint_data = twint.storage.panda.Tweets_df
  return twint_data

##Extraction des emojis

In [0]:
def extract_emojis(a_list):
    emojis_list = map(lambda x: ''.join(x.split()), emoji.UNICODE_EMOJI.keys())
    r = re.compile('|'.join(re.escape(p) for p in emojis_list))
    aux=[' '.join(r.findall(s)) for s in a_list]
    return(aux)

##Traduire les abréviations

In [0]:
def translator(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "slang.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    # Replacing commas with spaces for final output.
    return (' '.join(user_string))

##Cleaning du texte

In [0]:
def clean_text(text):
    """
    This function takes as input a text on which several 
    NLTK algorithms will be applied in order to preprocess it
    """
    # Remplacer les abréviations
    #print(text)
    #print(translator(text))
    text = translator(text)

    tokens = word_tokenize(text)
      
    # Remove the punctuations
    tokens = [word for word in tokens if word.isalpha()]
    # Lower the tokens
    tokens = [word.lower() for word in tokens]

    # Remove stopword
    tokens = [word for word in tokens if not word in stopwords.words("english")]

    # Corriger les fautes
    tokens = [str(TextBlob(word).correct()) for word in tokens]

    # Lemmatize
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word, pos = "v") for word in tokens]
    tokens = [lemma.lemmatize(word, pos = "n") for word in tokens]
    return tokens

#Execution du programme

In [0]:
def prog(mention,taille):
  Tweets_df = export_tweets(mention,taille)

  tweet_cleaned = []
  for i in range(len(Tweets_df)):
    tweet_cleaned.append(clean_text(Tweets_df['tweet'][i]))  

  emojisExtracted = extract_emojis(Tweets_df['tweet'])
  Tweets_df["tweet_emojis"] = emojisExtracted
  Tweets_df["tweet_cleaned"] = tweet_cleaned
  return Tweets_df

In [8]:
Tweets_df = prog("#ok",10)
#df.drop('column_name', axis=1, inplace=True)
Tweets_df

1244601539777495041 2020-03-30 12:25:05 UTC <DannysCarSale> Come test drive this 2008 Lincoln MKX! Get great mpg (city 16/ hwy 24) with only 160+ miles on the odometer! Find solace in ZERO accidents and a well documented service history! Call (580) 233-2277 today! #DannysCarSales #Enid #OK  pic.twitter.com/qUhwUJokEm
1244587774134693890 2020-03-30 11:30:23 UTC <BearWillLife> There's no better time than the present ! Individual, Team, and Family Coaching   http://ow.ly/uWgv50yZjL8  Wake Up ! #Lifecoah #Coach #TX #OK #NY #CA #Motivate #Determination #Improve #Life #DISC #Team #Teambuilding #Extreme #Execution #ExtremeExecution #CertifiedCoach #Chef #Food pic.twitter.com/E2piqGRSOb
1244576664505348096 2020-03-30 10:46:14 UTC <MayankYdv3> Gs que. Change  Most spoken word in the world before #OK Now #corona
1244575319912308737 2020-03-30 10:40:54 UTC <RobertPreuss>  https://www.technologyreview.com/s/615370/coronavirus-pandemic-social-distancing-18-months/ … OK. OK? #ok
1244571836370870273 

FileNotFoundError: ignored

#Idées

##Correction de sentences

In [0]:
from textblob import TextBlob

b = TextBlob("I havv goood speling!")
print(b.correct())

I have good spelling!


##Tokénisation

In [0]:
nltk.pos_tag(['go'])

[('go', 'VB')]

##Emotions *par mot*

In [0]:
from nltk.corpus import sentiwordnet as swn
list(swn.senti_synsets('love'))

breakdown = swn.senti_synset('kill.v.01')
print("pos_score: " , breakdown.pos_score())
print("neg_score: " , breakdown.neg_score())
print("obj_score: " , breakdown.obj_score())


pos_score:  0.0
neg_score:  0.5
obj_score:  0.5


##2eme essai de cleaning

In [0]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer 

#create an object of class PorterStemmer
stop_words = set(stopwords.words('english'))   

for i in range(len(Tweets_df)):
  word_tokens = word_tokenize(Tweets_df['tweet'][i])   
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  filtered_sentence = []   
  for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w) 
  #print(word_tokens) 
  #print(filtered_sentence)
  tweet_rmStop.append(filtered_sentence)
  
tweet_lemm = tweet_rmStop

In [0]:
# import these modules 
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

#create an object of class PorterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()

for i in range(len(tweet_rmStop)):
  for j in range(len(tweet_rmStop[i])):
    tweet_lemm[i][j] = porter.stem(tweet_rmStop[i][j])
    print(tweet_lemm[i][j])

In [0]:
import pandas
#Tweets_df = pandas.DataFrame()

Tweets_df["tweet_rmStop"]= tweet_rmStop
Tweets_df["tweet_lemm"] = tweet_lemm
Tweets_df["tweet_emojis"] = emojisExtracted